In [3]:
try:
    from google.colab import drive
except ModuleNotFoundError:
    IN_COLAB = False
else:
    drive.mount('/content/drive')
    IN_COLAB = True

In [4]:
if IN_COLAB:
    TRIPS_PATH = "/content/drive/MyDrive/Project/"
else:
    TRIPS_PATH = "../../data/"
TRIPS_PATH += "SK_pollution.csv"

In [8]:
trips.rename(columns={'Address': 'District', 'PM2.5': 'pm25'}, inplace=True)
trips['District'] = trips['District'].apply(lambda x: x.split(',')[2])
trips.head(1)

,Measurement date,Station code,District,Latitude,Longitude,SO2,NO2,O3,CO,PM10,pm25
0,2017-01-01 00:00,101,Jongno-gu,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0


In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

C:\Users\mohan\AppData\Local\Temp\ipykernel_11576\1820281231.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
trips = pd.read_csv(TRIPS_PATH)

In [7]:
gdf_trips = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.Longitude, trips.Latitude))
gdf_trips.set_crs('epsg:4326', inplace=True)

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,geometry
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,POINT (127.00501 37.57202)
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0,POINT (127.00501 37.57202)
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0,POINT (127.00501 37.57202)
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0,POINT (127.00501 37.57202)
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0,POINT (127.00501 37.57202)
...,...,...,...,...,...,...,...,...,...,...,...,...
647506,2019-12-31 19:00,125,"59, Gucheonmyeon-ro 42-gil, Gangdong-gu, Seoul...",37.544962,127.136792,0.003,0.028,0.013,0.5,23.0,17.0,POINT (127.13679 37.54496)
647507,2019-12-31 20:00,125,"59, Gucheonmyeon-ro 42-gil, Gangdong-gu, Seoul...",37.544962,127.136792,0.003,0.025,0.015,0.4,25.0,19.0,POINT (127.13679 37.54496)
647508,2019-12-31 21:00,125,"59, Gucheonmyeon-ro 42-gil, Gangdong-gu, Seoul...",37.544962,127.136792,0.003,0.023,0.015,0.4,24.0,17.0,POINT (127.13679 37.54496)
647509,2019-12-31 22:00,125,"59, Gucheonmyeon-ro 42-gil, Gangdong-gu, Seoul...",37.544962,127.136792,0.003,0.040,0.004,0.5,25.0,18.0,POINT (127.13679 37.54496)


In [9]:
df_pm25 = (trips
                          .groupby('District')
                          .pm25
                          .agg(list)
                          .to_frame("pm25_list")
                          .reset_index())# Let's count each points inside the hexagon

df_pm25['average_pm25_value'] =(df_pm25['pm25_list'].apply(lambda x:sum(x)/len(x)))
df_pm25.drop('pm25_list',axis = 1, inplace=True)

In [10]:
df_pm25.head()

,District,average_pm25_value
0,Dobong-gu,25.733277
1,Dongdaemun-gu,22.692596
2,Dongjak-gu,24.373021
3,Eunpyeong-gu,24.890639
4,Gangbuk-gu,21.726093


In [11]:
print(len(df_pm25["District"]), df_pm25["District"].nunique())

25 25


In [12]:
df_pm25.to_csv('../../data/Average pm25 per neighborhood BASELINE SK.csv')